In [1]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.preprocessing import image
from keras.initializers import glorot_uniform
import numpy as np

In [2]:
!pip install tflearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 7.4 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=079fa6b8668e30336d79a48fd67b06121c0e1669fcdd90c99660baeae4db5582
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [3]:
import tflearn.datasets.oxflower17 as oxflower17
x, y = oxflower17.load_data()

Instructions for updating:
non-resource variables are not supported in the long term


100.0% 60276736 / 60270631


Succesfully downloaded 17flowers.tgz 60270631 bytes.
File Extracted
Starting to parse images...
Parsing Done!


In [4]:
tf_flowers_labels = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']

In [5]:

import tensorflow as tf
import tensorflow_hub as hub

import tensorflow_datasets as tfds

import time

from PIL import Image
import requests
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

import os
import pathlib

In [6]:
NUM_CLASSES = 17
DATASET_NUM_TRAIN_EXAMPLES = 1360

In [28]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  "/content/17flowers/jpg",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(380, 380),
  batch_size=16)

Found 1360 files belonging to 17 classes.
Using 1088 files for training.


In [29]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  "/content/17flowers/jpg",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(380, 380),
  batch_size=16)

Found 1360 files belonging to 17 classes.
Using 272 files for validation.


In [24]:
base_model = tf.keras.applications.EfficientNetV2S(weights="imagenet", include_top=False, input_shape=(380,380,3))
base_model.trainable = False

In [20]:
for layer in base_model.layers[:100]:
   layer.trainable = False
for layer in base_model.layers[100:]:
   layer.trainable = True

In [25]:
from tensorflow.keras import layers, models

data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(380,
                                  380,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
  ]
)
closing_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = layers.Dense(17, activation='softmax')

bmodel = models.Sequential([
    data_augmentation,
    #tf.keras.layers.Rescaling(1./255),
    base_model,
    closing_layer,
    prediction_layer
])

In [11]:
train_callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5,
        patience=3, verbose=1
    )
]

In [26]:
bmodel.compile(optimizer='Adam',loss=tf.keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])

In [30]:
# Fine-tune model
history = bmodel.fit(
    train_ds,
    #batch_size=16,
    #steps_per_epoch=10,
    epochs= 50,  # TODO: replace with `epochs=10` here to shorten fine-tuning for tutorial if you wish
    validation_data=test_ds,  # here we are only using # this data to evaluate our performance
    callbacks=train_callbacks
    #validation_steps=5
)

Train on 68 steps, validate on 17 steps
Epoch 1/50
68/68 [==============================] - ETA: 0s - batch: 33.5000 - size: 1.0000 - loss: 1.7843 - acc: 0.5800

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


68/68 [==============================] - 25s 314ms/step - batch: 33.5000 - size: 1.0000 - loss: 1.7843 - acc: 0.5800 - val_loss: 1.1183 - val_acc: 0.7757 - lr: 0.0010
Epoch 2/50
68/68 [==============================] - 16s 235ms/step - batch: 33.5000 - size: 1.0000 - loss: 0.7919 - acc: 0.8759 - val_loss: 0.6862 - val_acc: 0.8529 - lr: 0.0010
Epoch 3/50
68/68 [==============================] - 18s 263ms/step - batch: 33.5000 - size: 1.0000 - loss: 0.5189 - acc: 0.9173 - val_loss: 0.5471 - val_acc: 0.8860 - lr: 0.0010
Epoch 4/50
68/68 [==============================] - 18s 263ms/step - batch: 33.5000 - size: 1.0000 - loss: 0.4005 - acc: 0.9338 - val_loss: 0.4713 - val_acc: 0.8934 - lr: 0.0010
Epoch 5/50
68/68 [==============================] - 17s 239ms/step - batch: 33.5000 - size: 1.0000 - loss: 0.3047 - acc: 0.9531 - val_loss: 0.4244 - val_acc: 0.8824 - lr: 0.0010
Epoch 6/50
68/68 [==============================] - 18s 265ms/step - batch: 33.5000 - size: 1.0000 - loss: 0.2524 - acc: 

In [32]:
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input, decode_predictions

In [34]:
import numpy as np

In [ ]:
img_path = '/content/17flowers/jpg/10/image_0801.jpg'
img = image.load_img(img_path, target_size=(380, 380))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
preds = bmodel.predict(x)
output = np.argmax(preds)
print(output)